In [1]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [2]:
import torch

def get_device():
    if torch.torch.cuda.is_available():
        return torch.device('cuda:0')
    else:
        return torch.device('cpu')

In [3]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModel
from datasets import load_dataset, Dataset, DatasetDict

### DETECTING PERSONAL INFORMATION WITH NER

#### USING DEFAULT PYTHAINLP

In [7]:
!pip install pythainlp

In [8]:
from pythainlp.tag import NER

In [9]:
sentence="ฉันชื่อ นางสาวมะลิวา บุญสระดี อาศัยอยู่ที่อำเภอนางรอง จังหวัดบุรีรัมย์ อายุ 23 ปี เพิ่งเรียนจบจาก มหาวิทยาลัยขอนแก่น และนี่คือข้อมูลปลอมชื่อคนไม่มีอยู่จริง อายุ 23 ปี"
ner = NER()
ner.tag(sentence)

[('ฉัน', 'O'),
 ('ชื่อ', 'O'),
 (' ', 'O'),
 ('นางสาว', 'B-PERSON'),
 ('มะลิ', 'I-PERSON'),
 ('วา', 'I-PERSON'),
 (' ', 'I-PERSON'),
 ('บุญ', 'I-PERSON'),
 ('สระ', 'I-PERSON'),
 ('ดี', 'I-PERSON'),
 (' ', 'O'),
 ('อาศัย', 'O'),
 ('อยู่', 'O'),
 ('ที่', 'O'),
 ('อําเภอ', 'B-LOCATION'),
 ('นาง', 'I-LOCATION'),
 ('รอง', 'I-LOCATION'),
 (' ', 'O'),
 ('จังหวัด', 'B-LOCATION'),
 ('บุรีรัมย์', 'I-LOCATION'),
 (' ', 'O'),
 ('อายุ', 'O'),
 (' ', 'O'),
 ('23', 'B-AGO'),
 (' ', 'I-AGO'),
 ('ปี', 'I-AGO'),
 (' ', 'O'),
 ('เพิ่ง', 'O'),
 ('เรียนจบ', 'O'),
 ('จาก', 'O'),
 (' ', 'O'),
 ('มหาวิทยาลั', 'B-ORGANIZATION'),
 ('ยขอนแก่น', 'I-ORGANIZATION'),
 (' ', 'O'),
 ('และ', 'O'),
 ('นี่', 'O'),
 ('คือ', 'O'),
 ('ข้อมูล', 'O'),
 ('ปลอม', 'O'),
 ('ชื่อ', 'O'),
 ('คน', 'O'),
 ('ไม่', 'O'),
 ('มี', 'O'),
 ('อยู่', 'O'),
 ('จริง', 'O'),
 (' ', 'O'),
 ('อายุ', 'O'),
 (' ', 'O'),
 ('23', 'B-AGO'),
 (' ', 'O'),
 ('ปี', 'I-AGO')]

#### IN CASE OF USING CUSTOM TOKENIZER

In [10]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
from pythainlp.tokenize import word_tokenize # pip install pythainlp
import torch

In [15]:
name="pythainlp/thainer-corpus-v2-base-model"
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForTokenClassification.from_pretrained(name)

sentence="ฉันชื่อ นางสาวมะลิวา บุญสระดี อาศัยอยู่ที่อำเภอนางรอง จังหวัดบุรีรัมย์ อายุ 23 ปี เพิ่งเรียนจบจาก มหาวิทยาลัยขอนแก่น และนี่คือข้อมูลปลอมชื่อคนไม่มีอยู่จริง อายุ 23 ปี"

# replace space (" ") with "<_>"
sentence = sentence.replace(" ", "<_>")
# custom tokenizer can be used here
cut=word_tokenize(sentence
                  )
inputs=tokenizer(cut,is_split_into_words=True,return_tensors="pt")

ids = inputs["input_ids"]
mask = inputs["attention_mask"]
# forward pass
outputs = model(ids, attention_mask=mask)
logits = outputs[0]

predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

def fix_span_error(words,ner):
    _ner = []
    _ner=ner
    _new_tag=[]
    for i,j in zip(words,_ner):
        #print(i,j)
        i=tokenizer.decode(i)
        if i.isspace() and j.startswith("B-"):
            j="O"
        if i=='' or i=='<s>' or i=='</s>':
            continue
        if i=="<_>":
            i=" "
        _new_tag.append((i,j))
    return _new_tag

ner_tag=fix_span_error(inputs['input_ids'][0],predicted_token_class)

ner_tag

[('ฉัน', 'O'),
 ('ชื่อ', 'O'),
 (' ', 'O'),
 ('นางสาว', 'B-PERSON'),
 ('มะลิ', 'I-PERSON'),
 ('วา', 'I-PERSON'),
 (' ', 'I-PERSON'),
 ('บุญ', 'I-PERSON'),
 ('สระ', 'I-PERSON'),
 ('ดี', 'I-PERSON'),
 (' ', 'O'),
 ('อาศัย', 'O'),
 ('อยู่', 'O'),
 ('ที่', 'O'),
 ('อําเภอ', 'B-LOCATION'),
 ('นาง', 'I-LOCATION'),
 ('รอง', 'I-LOCATION'),
 (' ', 'O'),
 ('จังหวัด', 'B-LOCATION'),
 ('บุรีรัมย์', 'I-LOCATION'),
 (' ', 'O'),
 ('อายุ', 'O'),
 (' ', 'O'),
 ('23', 'B-AGO'),
 (' ', 'I-AGO'),
 ('ปี', 'I-AGO'),
 (' ', 'O'),
 ('เพิ่ง', 'O'),
 ('เรียนจบ', 'O'),
 ('จาก', 'O'),
 (' ', 'O'),
 ('มหาวิทยาลั', 'B-ORGANIZATION'),
 ('ยขอนแก่น', 'I-ORGANIZATION'),
 (' ', 'O'),
 ('และ', 'O'),
 ('นี่', 'O'),
 ('คือ', 'O'),
 ('ข้อมูล', 'O'),
 ('ปลอม', 'O'),
 ('ชื่อ', 'O'),
 ('คน', 'O'),
 ('ไม่', 'O'),
 ('มี', 'O'),
 ('อยู่', 'O'),
 ('จริง', 'O'),
 (' ', 'O'),
 ('อายุ', 'O'),
 (' ', 'O'),
 ('23', 'B-AGO'),
 (' ', 'O'),
 ('ปี', 'I-AGO')]